In [4]:
# %% ---------------- centre-aligned, over-scaled media replacement ----------------
import cv2, json, numpy as np, logging, math, pathlib

# ── files ──────────────────────────────────────────────────────────────
TRACK_JSON = "aruco_screen_tracks.json"   # produced by the tracker
VIDEO_IN   = "aruco.mp4"                  # original recording
MEDIA_IMG  = "media.png"                  # picture to show on screen
VIDEO_OUT  = "screen_with_media.mp4"      # result
ALPHA      = 0.0                          # 0 = full replace; >0 = faint underlay

# ── debug log (optional) ───────────────────────────────────────────────
logging.basicConfig(filename="replace_debug.log",
                    filemode="w", level=logging.DEBUG, format="%(message)s")
log = logging.getLogger("replace")

# ═══ 1.  load tracking JSON ════════════════════════════════════════════
with open(TRACK_JSON, "r", encoding="utf-8") as f:
    meta = json.load(f)

frames = meta["frames"]
FW, FH = meta["frame_width"], meta["frame_height"]

# ═══ 2.  open video & media image ═════════════════════════════════════
cap  = cv2.VideoCapture(VIDEO_IN)
fps  = cap.get(cv2.CAP_PROP_FPS)
four = cv2.VideoWriter_fourcc(*"mp4v")
out  = cv2.VideoWriter(VIDEO_OUT, four, fps, (FW, FH))

media = cv2.imread(MEDIA_IMG)
if media is None:
    raise RuntimeError(f"Cannot read {MEDIA_IMG}")
mh, mw = media.shape[:2]

# reusable 4-point source quad
src_quad = np.zeros((4, 2), np.float32)

# ═══ 3.  frame loop ═══════════════════════════════════════════════════
for rec in frames:
    ok, frame = cap.read()
    if not ok:
        break

    if not rec["valid"]:
        out.write(frame)
        continue

    dst_quad = np.array(rec["corners"], np.float32)   # TL,TR,BR,BL

    # overlay size this frame
    ov_w = int(np.linalg.norm(dst_quad[1] - dst_quad[0]) + 0.5)   # top edge
    ov_h = int(np.linalg.norm(dst_quad[3] - dst_quad[0]) + 0.5)   # left edge

    # -------- always overscale *at least* one pixel larger -----------
    scale = max(ov_w / mw, ov_h / mh) * 1.01        # +1 % safety
    big   = cv2.resize(media,
                       (math.ceil(mw * scale), math.ceil(mh * scale)),
                       interpolation=cv2.INTER_AREA)
    bh, bw = big.shape[:2]

    # centred crop to exact overlay size (keeps centre of mass)
    x0 = max(0, (bw - ov_w) // 2)
    y0 = max(0, (bh - ov_h) // 2)
    patch = big[y0:y0 + ov_h, x0:x0 + ov_w]

    # source quad matches patch
    ph, pw = patch.shape[:2]        # pw==ov_w , ph==ov_h
    src_quad[:] = [[0, 0], [pw - 1, 0], [pw - 1, ph - 1], [0, ph - 1]]

    # -------- warp & composite ---------------------------------------
    H      = cv2.getPerspectiveTransform(src_quad, dst_quad)
    warped = cv2.warpPerspective(patch, H, (FW, FH))

    mask   = cv2.warpPerspective(np.full((ph, pw), 255, np.uint8), H, (FW, FH))
    mask_i = cv2.bitwise_not(mask)

    if ALPHA:
        base = cv2.addWeighted(frame, 1 - ALPHA, warped, ALPHA, 0)
        frame = cv2.bitwise_and(base, base, mask=mask) + cv2.bitwise_and(frame, frame, mask=mask_i)
    else:
        frame = cv2.bitwise_and(frame, frame, mask=mask_i) + cv2.bitwise_and(warped, warped, mask=mask)

    out.write(frame)

cap.release(); out.release()
print("✅  Media burned in – saved to", VIDEO_OUT)


✅  Media burned in – saved to screen_with_media.mp4
